# Lecon 2 : Apprendre de ses erreurs

## Le secret de l'IA : se tromper, corriger, recommencer

Imagine que tu apprends a lancer une balle dans un panier :
1. Tu lances -> tu rates a droite
2. Tu corriges un peu a gauche
3. Tu relances -> plus pres !
4. Tu continues jusqu'a marquer

L'IA fait **exactement** pareil. Elle fait une prediction, regarde si c'est
bon, et ajuste. Ca s'appelle **l'entrainement**.

## Etape 1 : Mesurer l'erreur

D'abord, il faut un moyen de dire **a quel point** le modele s'est trompe.
On appelle ca la **loss** (perte en anglais).

- Loss haute = le modele se trompe beaucoup
- Loss basse = le modele devine bien

In [ ]:
import math

# Imaginons que le modele predit les probabilites suivantes
# pour la lettre qui suit 'h' dans le prenom 'hugo' :

prediction = {
    'u': 0.6,   # 60% -> bonne reponse !
    'a': 0.2,   # 20%
    'e': 0.15,  # 15%
    'o': 0.05,  # 5%
}

# La bonne reponse est 'u'
bonne_reponse = 'u'

# La loss = a quel point on est surpris par la bonne reponse
# Si on avait dit 100% pour 'u', la surprise serait de 0 (parfait !)
# Si on avait dit 1% pour 'u', la surprise serait enorme

loss = -math.log(prediction[bonne_reponse])
print(f"Le modele donnait {prediction[bonne_reponse]:.0%} de chance a '{bonne_reponse}'")
print(f"Loss = {loss:.2f}")
print()

# Comparons avec une mauvaise prediction
mauvaise_prediction = {'u': 0.05, 'a': 0.7, 'e': 0.2, 'o': 0.05}
loss_mauvaise = -math.log(mauvaise_prediction[bonne_reponse])
print(f"Si le modele n'avait donne que {mauvaise_prediction[bonne_reponse]:.0%} a '{bonne_reponse}'...")
print(f"Loss = {loss_mauvaise:.2f}  (beaucoup plus haut = beaucoup plus faux)")

## Etape 2 : Les poids du modele

Un modele, c'est juste une collection de **nombres** (on les appelle des **poids**).
Ces nombres determinent les predictions.

Entrainer = trouver les bons nombres.

In [ ]:
import random
import math

# On cree un mini-modele : juste des scores pour chaque paire de lettres
# Au debut, les scores sont aleatoires -> le modele ne sait rien

alphabet = list("abcdefghijklmnopqrstuvwxyz.")

# Scores aleatoires (les "poids" du modele)
random.seed(42)
poids = {}
for a in alphabet:
    poids[a] = {}
    for b in alphabet:
        poids[a][b] = random.uniform(-1, 1)

def calculer_probas(poids, lettre):
    """Transforme les scores en probabilites (softmax)."""
    scores = poids[lettre]
    # L'exponentielle rend tous les scores positifs
    exps = {b: math.exp(scores[b]) for b in scores}
    total = sum(exps.values())
    return {b: exps[b] / total for b in scores}

# Au debut, les probas sont quasi uniformes (le modele devine au hasard)
p = calculer_probas(poids, '.')
lettres_debut = sorted(p.items(), key=lambda x: -x[1])[:5]
print("Au debut, le modele pense que les prenoms commencent par :")
for lettre, prob in lettres_debut:
    print(f"  '{lettre}' : {prob:.1%}")
print("\n  -> C'est n'importe quoi ! Il faut l'entrainer.")

## Etape 3 : Entrainement

L'algorithme est simple :
1. Prendre un prenom d'entrainement
2. Le modele fait sa prediction
3. On calcule la loss (l'erreur)
4. On **ajuste les poids** pour reduire la loss
5. Recommencer

L'etape 4 s'appelle la **descente de gradient**. C'est comme ajuster ton
tir au panier un petit peu a chaque essai.

In [ ]:
prenoms = [
    "emma", "lucas", "lea", "hugo", "chloe",
    "louis", "alice", "jules", "lina", "adam",
    "rose", "arthur", "manon", "paul", "jade",
    "nathan", "eva", "leo", "clara", "noah",
]

# Vitesse d'apprentissage : de combien on ajuste a chaque fois
# Trop grand = on depasse, trop petit = on apprend trop lentement
vitesse = 0.1

print("Entrainement...")
print()

for epoch in range(50):
    loss_totale = 0
    nb = 0

    for prenom in prenoms:
        mot = "." + prenom + "."
        for i in range(len(mot) - 1):
            lettre = mot[i]
            cible = mot[i + 1]

            # 1. Prediction
            probas = calculer_probas(poids, lettre)

            # 2. Loss
            loss_totale += -math.log(probas[cible] + 1e-10)
            nb += 1

            # 3. Ajuster les poids (gradient simplifie)
            for b in alphabet:
                if b == cible:
                    # La bonne reponse : augmenter son score
                    poids[lettre][b] += vitesse * (1 - probas[b])
                else:
                    # Les mauvaises reponses : baisser leur score
                    poids[lettre][b] -= vitesse * probas[b]

    if epoch % 10 == 0:
        print(f"  Epoch {epoch:2d} | Loss moyenne : {loss_totale / nb:.3f}")

print(f"  Epoch {epoch:2d} | Loss moyenne : {loss_totale / nb:.3f}")
print()
print("La loss baisse = le modele s'ameliore !")

In [ ]:
# Voyons maintenant ce que le modele a appris :
p = calculer_probas(poids, '.')
lettres_debut = sorted(p.items(), key=lambda x: -x[1])[:5]
print("Apres entrainement, les prenoms commencent par :")
for lettre, prob in lettres_debut:
    print(f"  '{lettre}' : {prob:.1%}")
print()
print("C'est plus logique ! (l, a, e, c, j sont des debuts courants)")

In [ ]:
# Generons des prenoms avec le modele entraine
def generer(poids, n=10):
    resultats = []
    for _ in range(n):
        prenom = ""
        lettre = "."
        for _ in range(20):  # max 20 lettres
            p = calculer_probas(poids, lettre)
            choix = list(p.keys())
            probs = list(p.values())
            lettre = random.choices(choix, weights=probs, k=1)[0]
            if lettre == ".":
                break
            prenom += lettre
        if prenom:
            resultats.append(prenom.capitalize())
    return resultats

print("Prenoms inventes apres entrainement :")
for p in generer(poids, 10):
    print(f"  {p}")

## Ce qu'on a appris

- La **loss** mesure a quel point le modele se trompe
- Les **poids** sont les nombres que le modele ajuste pour apprendre
- L'**entrainement** = ajuster les poids pour reduire la loss, encore et encore
- Meme un modele simple s'ameliore avec l'entrainement !

### Limite

Notre modele ne regarde encore que **1 lettre en arriere**.
Dans la prochaine lecon, on va lui donner une **memoire** pour qu'il
se souvienne de plusieurs lettres a la fois.

---
*Prochaine lecon : [03 - La memoire du modele](03_la_memoire_du_modele.ipynb)*